In [1]:
import dgl
import torch


In [4]:
load_path = "/data/sandeep/groot_data/snap/com-friendster"
partition_map = torch.load(f"{load_path}/partition_map_True").to(0)
indptr = torch.load(f"{load_path}/indptr_64_wsloop.pt")
indices = torch.load(f"{load_path}/indices_64_wsloop.pt")
train_idx = torch.load(f"{load_path}/train_idx_64.pt").to(0)
edges = torch.empty(0, dtype=indices.dtype)


In [5]:
graph = dgl.graph(('csc', (indptr,indices,edges)))
graph.pin_memory_()

Graph(num_nodes=124836180, num_edges=1930903315,
      ndata_schemes={}
      edata_schemes={})

In [6]:
indptr_d = indptr.to(0)

In [7]:
num_nodes = indptr.shape[0] - 1

def final_freq_per_partition(training_ids, graph, partition_map):
    device = 0
    use_uva = True
    batch_size = 1024
    sampler = dgl.dataloading.NeighborSampler([20,20,20])
    dataloader = dgl.dataloading.DataLoader(graph = graph,
                                            indices = training_ids.to(device),
                                            graph_sampler = sampler,
                                            use_uva=use_uva,
                                            batch_size=batch_size)
    sample_freq = torch.zeros(num_nodes, dtype=torch.int32, device=device )
    for batch_input, batch_seeds, batch_blocks in dataloader:
        sample_freq[batch_input] += 1
    select_nodes = torch.where(sample_freq != 0)[0]
    for i in range(4):
        print(i,torch.sum(partition_map[select_nodes] == i))
        

In [8]:
train_idx1 = train_idx[partition_map[train_idx] == 0]
print(train_idx1.shape)
final_freq_per_partition(train_idx1, graph, partition_map)

torch.Size([349873])
0 tensor(7449538, device='cuda:0')
1 tensor(3817081, device='cuda:0')
2 tensor(3097681, device='cuda:0')
3 tensor(2235719, device='cuda:0')


In [12]:
final_idx = []
for i in range(4):
    original_train_idx = train_idx[partition_map[train_idx] == i]
    select = torch.where(partition_map == i)[0]
    degrees = indptr_d[select + 1] - indptr_d[select]
    _, inv = torch.sort(degrees, descending = True )
    select = inv[:original_train_idx.shape[0]]
    final_idx.append(select)

training_alt = torch.cat(final_idx,dim = 0)

    # print(select.shape)
# final_freq_per_partition(select, graph, partition_map)

In [13]:
torch.save(training_alt,f"{load_path}/training_alt.pt")